In [31]:
import re
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from gensim.test.utils import common_corpus, common_dictionary
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath
from gensim.models import LdaModel

with open('chunk_8.txt', 'r', encoding='latin-1') as f:
    clean_cont = f.read().splitlines()

doc=[i.replace('\xe2\x80\x9c','') for i in clean_cont ]
doc=[i.replace('\xe2\x80\x9d','') for i in doc ]
doc=[i.replace('\xe2\x80\x99s','') for i in doc ]
docs = [x for x in doc if x != ' ']
docss = [x for x in docs if x != '']
financedoc=[re.sub("[^a-zA-Z]+", " ", s) for s in docss]

# Tokenize the preprocessed text documents
tokenized_docs = [doc.split() for doc in financedoc]
# Define English stopwords
english_stopwords = set(stopwords.words('english'))
# Define additional words to include as stopwords
additional_stopwords = {"A", "B", "C","S" ,"a", "b", "c", "s","Total","Period","Other","ended", "Whether", "wells", "Fargo", "Wells"}
# Add the additional stopwords to the set
english_stopwords.update(additional_stopwords)

# Step 1: Remove English stopwords during tokenization
tokenized_docs_no_stopwords = [[word for word in doc if word not in english_stopwords] for doc in tokenized_docs]

# Step 2: Create a Gensim dictionary
gensim_dictionary = Dictionary(tokenized_docs_no_stopwords)

# Optionally: Filter out very rare and very common words
gensim_dictionary.filter_extremes(no_below=5, no_above=0.5)  # Adjust as needed

# Step 3: Convert tokenized documents into the Gensim corpus format
corpus = [gensim_dictionary.doc2bow(doc) for doc in tokenized_docs_no_stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yuany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
lda = LdaMulticore(corpus, id2word=gensim_dictionary, num_topics=5)
benchmark_model = datapath("benchmark_model")
lda.save(benchmark_model)
lda.print_topics(5)

[(0,
  '0.056*"financial" + 0.044*"liabilities" + 0.036*"billion" + 0.029*"income" + 0.029*"September" + 0.025*"assets" + 0.025*"consolidated" + 0.025*"entities" + 0.022*"We" + 0.022*"loans"'),
 (1,
  '0.058*"securities" + 0.032*"Securities" + 0.029*"loans" + 0.026*"U" + 0.023*"stock" + 0.020*"equity" + 0.020*"fair" + 0.019*"millions" + 0.017*"value" + 0.016*"federal"'),
 (2,
  '0.055*"million" + 0.036*"Net" + 0.035*"accounting" + 0.032*"value" + 0.032*"change" + 0.031*"securities" + 0.031*"statements" + 0.025*"fair" + 0.024*"The" + 0.021*"interests"'),
 (3,
  '0.057*"stock" + 0.048*"securities" + 0.035*"term" + 0.029*"net" + 0.029*"assets" + 0.026*"Common" + 0.023*"investments" + 0.023*"We" + 0.023*"debt" + 0.023*"shares"'),
 (4,
  '0.052*"The" + 0.038*"Net" + 0.032*"billion" + 0.029*"cash" + 0.027*"activities" + 0.026*"net" + 0.026*"losses" + 0.026*"ASU" + 0.023*"assets" + 0.023*"stock"')]

In [33]:
print(gensim_dictionary[1])

Subsidiaries


__Load the Mapping of Lower-Cased Vocabulary Items to Their Most Common Surface Form__

In [34]:
import os
import numpy
# Load surface form mappings here

current_directory = os.getcwd()
filename="Vocab2SurfaceFormMapping.tsv"
fp = open(os.path.join(current_directory, filename))    

vocabToSurfaceFormHash = {}

# Each line in the file has two tab separated fields;
# the first is the vocabulary item used during modeling
# and the second is its most common surface form in the 
# original data
for stringIn in fp.readlines():
    fields = stringIn.strip().split("\t")
    if len(fields) != 2:
        print ("Warning: Bad line in surface form mapping file: %s" % stringIn)
    elif fields[0] == "" or fields[1] == "":
        print ("Warning: Bad line in surface form mapping file: %s" % stringIn)
    else:
        vocabToSurfaceFormHash[fields[0]] = fields[1]
fp.close()
print(vocabToSurfaceFormHash)

{'Surface_Form': 'Modeled_Term', 'income statement': 'financial_statements', 'statement of earnings': 'financial_statements', 'profit and loss statement': 'financial_statements', 'balance sheet              ': 'financial_statements', 'income statement           ': 'financial_statements', 'P&L statement              ': 'financial_statements', 'Comprehensive Income': 'financial_statements', 'Changes in Equity': 'financial_statements', 'Consolidated Statement': 'financial_statements', 'securities': 'shares'}


In [35]:
def CreateTermIDToSurfaceFormMapping(id2token, token2surfaceform):
    termIDToSurfaceFormMap = []
    for i in range(0, len(id2token)):
        if id2token[i] in token2surfaceform:
            termIDToSurfaceFormMap.append(token2surfaceform[id2token[i]])
        else:
             termIDToSurfaceFormMap.append(id2token[i])

    return termIDToSurfaceFormMap;

termIDToSurfaceFormMap = CreateTermIDToSurfaceFormMapping(gensim_dictionary, vocabToSurfaceFormHash);


print(gensim_dictionary)
# print out the modeled token form and the best matching surface for the token with the index value of 18
i = 1
print('Term index:', i)
print('Surface form:', gensim_dictionary[i])
print('Modeled form:', termIDToSurfaceFormMap[i])

Dictionary<140 unique tokens: ['Company', 'Subsidiaries', 'Consolidated', 'Statement', 'millions']...>
Term index: 1
Surface form: Subsidiaries
Modeled form: Subsidiaries


In [36]:

# To retrieve all topics and their probabilities we must set the LDA minimum probability setting to zero
lda.minimum_probability = 0

# This function generates the topic probabilities for each doc from the trained LDA model
# The probabilities are placed in a single matrix where the rows are documents and columns are topics
def ExtractDocTopicProbsMatrix(corpus,lda):
    # Initialize the matrix
    docTopicProbs = numpy.zeros((len(corpus),lda.num_topics))
    for docID in range(0,len(corpus)):
        for topicProb in lda[corpus[docID]]:
            docTopicProbs[docID,topicProb[0]]=topicProb[1]
    return docTopicProbs    

# docTopicProbs[docID,TopicID] --> P(topic|doc)
docTopicProbs = ExtractDocTopicProbsMatrix(corpus, lda)

docTopicProbs.shape

(1299, 5)

__Compute the Global Topic Likelihood Scores P(topic)__

In [37]:
# Computing the global topic likelihoods by aggregating topic probabilities over all documents
# topicProbs[topicID] --> P(topic)
def ComputeTopicProbs(docTopicProbs):
    topicProbs = docTopicProbs.sum(axis=0) 
    topicProbs = topicProbs/sum(topicProbs)
    return topicProbs

topicProbs = ComputeTopicProbs(docTopicProbs)
print(topicProbs)

[0.20540022 0.19566252 0.19137388 0.20977199 0.19779139]


__Convert the Topic Language Model Information P(term|topic) from the LDA Model into a NumPy Representation__

In [38]:
def ExtractTopicLMMatrix(lda):
    # Initialize the matrix
    docTopicProbs = numpy.zeros((lda.num_topics,lda.num_terms))
    for topicID in range(0,lda.num_topics):
        termProbsList = lda.get_topic_terms(topicID,lda.num_terms)
        for termProb in termProbsList:
            docTopicProbs[topicID,termProb[0]]=termProb[1]
    return docTopicProbs
    
# topicTermProbs[topicID,termID] --> P(term|topic)
topicTermProbs = ExtractTopicLMMatrix(lda)
print(topicTermProbs)

[[0.01459535 0.00074783 0.00075402 0.00416383 0.00081808 0.02861062
  0.00430825 0.01811735 0.00769313 0.00770886 0.01261785 0.00085736
  0.00422088 0.00078074 0.01465043 0.00075649 0.00077692 0.00078048
  0.00424528 0.00768563 0.0007275  0.01109676 0.00770423 0.00421305
  0.00772286 0.00429639 0.00074515 0.01815366 0.00076806 0.0008281
  0.00426105 0.01575166 0.00884713 0.00905604 0.00193495 0.01937461
  0.00170668 0.00074355 0.00523751 0.01567702 0.00072523 0.00183846
  0.00429843 0.02530282 0.0112521  0.00079374 0.01125694 0.00769996
  0.00774281 0.00072087 0.00075013 0.00073867 0.00768327 0.0007645
  0.02161301 0.00076969 0.00414628 0.04373571 0.01123456 0.00442719
  0.00080819 0.00102779 0.00079754 0.00075343 0.01458634 0.01117739
  0.00075263 0.00076337 0.00423491 0.01463752 0.00772596 0.02856942
  0.0076986  0.02512334 0.02504873 0.02149512 0.00423174 0.0133269
  0.03556632 0.00414146 0.00074027 0.00073912 0.00075677 0.00074083
  0.00430974 0.00779639 0.00432736 0.0042165  0.004

__Compute P(topic,term), P(term), and P(topic|term)__

In [39]:
# Compute the joint likelihoods of topics and terms
# jointTopicTermProbs[topicID,termID] --> P(topic,term) = P(term|topic)*P(topic)
jointTopicTermProbs = numpy.diag(topicProbs).dot(topicTermProbs) 

# termProbs[termID] --> P(term)
termProbs = jointTopicTermProbs.sum(axis=0)

# topicProbsPermTerm[topicID,termID] --> P(topic|term)
topicProbsPerTerm = jointTopicTermProbs / termProbs

In [40]:
# Print most frequent words in LDA vocab
mostFrequentTermIDs = (-termProbs).argsort()
for i in range(0,25):
    print ("%d: %s --> %f" % (i+1, gensim_dictionary[mostFrequentTermIDs[i]], termProbs[mostFrequentTermIDs[i]]))

1: securities --> 0.033853
2: stock --> 0.022746
3: assets --> 0.018656
4: financial --> 0.018643
5: income --> 0.018046
6: The --> 0.017285
7: billion --> 0.016681
8: Net --> 0.016107
9: million --> 0.015502
10: value --> 0.014646
11: net --> 0.014341
12: We --> 0.014106
13: September --> 0.013511
14: fair --> 0.013260
15: ASU --> 0.012918
16: statements --> 0.012148
17: loans --> 0.011883
18: Cash --> 0.011844
19: change --> 0.011832
20: equity --> 0.011808
21: debt --> 0.011373
22: liabilities --> 0.010603
23: accounting --> 0.010102
24: Note --> 0.009997
25: sale --> 0.009890


__Compute WPMI__  


In [41]:
topicTermWPMI =(jointTopicTermProbs.T * numpy.log(topicProbsPerTerm.T / topicProbs)).T
topicTermWPMI.shape

(5, 140)

__Compute Topic to Document Purity measure for Each Topic__  


In [42]:
topicPurity = numpy.exp(((docTopicProbs * numpy.log(docTopicProbs)).sum(axis=0))/(docTopicProbs).sum(axis=0))

__Create Topic Summaries__  


In [43]:
topicID = 2

highestWPMITermIDs = (-topicTermWPMI[topicID]).argsort()
highestProbTermIDs = (-topicTermProbs[topicID]).argsort()
print ("                                        WPMI                                                 Prob")
for i in range(0,15):
    print ("%2d: %35s ---> %8.6f    %35s ---> %8.6f" % (i+1, 
                                                        termIDToSurfaceFormMap[highestWPMITermIDs[i]], 
                                                        topicTermWPMI[topicID,highestWPMITermIDs[i]],
                                                        termIDToSurfaceFormMap[highestProbTermIDs[i]], 
                                                        topicTermProbs[topicID,highestProbTermIDs[i]]))       

                                        WPMI                                                 Prob
 1:                             million ---> 0.013120                                million ---> 0.054517
 2:                          accounting ---> 0.008517                                    Net ---> 0.035574
 3:                              change ---> 0.006115                             accounting ---> 0.035451
 4:                          statements ---> 0.005402                                  value ---> 0.032106
 5:                                 Net ---> 0.005394                                 change ---> 0.032057
 6:                               value ---> 0.004823                                 shares ---> 0.031431
 7:                           interests ---> 0.004393                             statements ---> 0.030577
 8:                           available ---> 0.003542                                   fair ---> 0.025162
 9:                        compensation ---> 0

In [44]:
def CreateTopicSummaries(topicTermScores, id2token, tokenid2surfaceform, maxStringLen):
    reIgnore = re.compile('^[a-z]\.$')
    reAcronym = re.compile('^[A-Z]+$')
    topicSummaries = []
    for topicID in range(0,len(topicTermScores)):
        rankedTermIDs = (-topicTermScores[topicID]).argsort()
        maxNumTerms = len(rankedTermIDs)
        termIndex = 0
        stop = 0
        outputTokens = []
        prevAcronyms = []
        topicSummary = ""
        while not stop:
            # If we've run out of tokens then stop...
            if (termIndex>=maxNumTerms):
                stop=1
            # ...otherwise consider adding next token to summary
            else:
                nextToken = id2token[rankedTermIDs[termIndex]]
                nextTokenOut = tokenid2surfaceform[rankedTermIDs[termIndex]]
                keepToken = 1
                
                # Prepare to test current word as an acronym or a string that reduces to an acronym
                nextTokenIsAcronym = 0
                nextTokenAbbrev = ""
                if reAcronym.match(nextTokenOut) != None:
                    nextTokenIsAcronym = 1
                else:
                    subTokens = nextToken.split('_')
                    if (len(subTokens)>1):
                        for subToken in subTokens:
                            nextTokenAbbrev += subToken[0]                        

                # See if we should ignore this token because it matches the regex for tokens to ignore
                if ( reIgnore.match(nextToken) != None ):
                    keepToken = 0;

                # Otherwise see if we should ignore this token because
                # it is a close match to a previously selected token
                elif len(outputTokens) > 0:          
                    for prevToken in outputTokens:
                        # Ignore token if it is a substring of a previous token
                        if nextToken in prevToken:
                            keepToken = 0
                        # Ignore token if it is a superstring of a previous token
                        elif prevToken in nextToken:
                            keepToken = 0
                        # Ignore token if it is an acronym of a previous token
                        elif nextTokenIsAcronym:
                            subTokens = prevToken.split('_')
                            if (len(subTokens)>1):
                                prevTokenAbbrev = ""
                                for subToken in subTokens:
                                    prevTokenAbbrev += subToken[0]
                                if prevTokenAbbrev == nextToken:
                                    keepToken = 0                                  
                    for prevAcronym in prevAcronyms:
                        # Ignore token if it is the long form of an earlier acronym
                        if nextTokenAbbrev == prevAcronym:
                                keepToken = 0

                # Add tokens to the summary for this topic                
                if keepToken:
                    # Always add at least one token to the summary
                    if len(topicSummary) == 0 or ( len(topicSummary) + len(nextTokenOut) + 1 < maxStringLen):
                        if len(topicSummary) == 0:
                            topicSummary = nextTokenOut
                        else: 
                            topicSummary += ", " + nextTokenOut
                        outputTokens.append(nextToken)
                        if nextTokenIsAcronym:
                            prevAcronyms.append(nextToken)
                    # If we didn't add the previous word and we're within 10 characters of 
                    # the max string length then we'll just stop here
                    elif maxStringLen - len(topicSummary) < 10 :
                        stop = 1
                    # Otherwise if the current token is too long, but we still have more than
                    # 10 characters of space left we'll just skip this one and add the next token
                    # one if it's short enough
                termIndex += 1         
        topicSummaries.append(topicSummary)
    return topicSummaries   
    
topicSummaries = CreateTopicSummaries(topicTermWPMI, gensim_dictionary, gensim_dictionary, 85)

In [45]:
# Rank the topics by their prominence score in the corpus
# The topic score combines the total weight of each a topic in the corpus 
# with a topic document purity score for topic 
# Topics with topicScore > 1 are generally very strong topics

numTopics =5
topicScore = (numTopics * topicProbs) * (2 * topicPurity)
topicRanking = (-topicScore).argsort()

print ("Rank  ID  Score  Prob  Purity  Summary")
for i in range(0, numTopics):
    topicID = topicRanking[i]
    print (" %3d %3d %6.3f (%5.3f, %4.3f) %s" 
           % (i+1, topicID, topicScore[topicID], 100*topicProbs[topicID], topicPurity[topicID], topicSummaries[topicID]))

Rank  ID  Score  Prob  Purity  Summary
   1   3  0.575 (20.977, 0.274) stock, term, Common, investments, short, issued, shares, net, securities, loss, debt
   2   0  0.548 (20.540, 0.267) financial, liabilities, entities, billion, interest, consolidated, Accounting
   3   4  0.507 (19.779, 0.257) The, activities, cash, losses, Net, gains, acquired, application, mortgage, deposits
   4   1  0.493 (19.566, 0.252) Securities, U, securities, loans, millions, agencies, federal, states, additional
   5   2  0.466 (19.137, 0.244) million, accounting, change, statements, Net, value, interests, available, certain
